In [1]:
import pandas as pd

In [2]:
def combine_two_part_ep(two_part_ep):
    for ep in two_part_ep:
        join_ep = []
        path_1 = ("S1/C1E%s-1_FINAL.txt" % ep)
        path_2 = ("S1/C1E%s-2_FINAL.txt" % ep)
        with open(path_1, "r", encoding="latin-1") as p1_ep:
            p1_ep = p1_ep.read().splitlines()
            with open(path_2, "r", encoding="latin-1") as p2_ep:
                p2_ep = p2_ep.read().splitlines()
                join_ep = p1_ep + p2_ep
        with open("S1/C1E%s_FINAL.txt" % ep, 'w', encoding='latin-1') as comb_ep:
            for line in join_ep:
                comb_ep.write('%s\n' % line)

combine_two_part_ep(['035', '033', '031'])

In [32]:
def split_name_speech_count(cont_line):
    name_split = cont_line.index(':')
    name = cont_line[:name_split]

    speak_split = name_split + 2
    speak = cont_line[speak_split:]

    count = len(speak.split())

    split_line = (name, speak, count)
    return split_line


def extract_lines(raw_lines):
    count = 0
    org_list = []
    cont_line = []
    raw_lines = [t for t in raw_lines if t]

    for line in raw_lines:
        line = line.lower()
        remove_char = ['\'', '\"', '.', ',', '?', '/', '!']
        for char in remove_char:
            line = line.replace(char, '')

        count += 1
        if count != len(raw_lines):
            if ':' in line:
                colen_index= line.index(':')
                if colen_index <= 30:
                    if cont_line != []:
                        split_line = split_name_speech_count(cont_line)
                        org_list.append(split_line)
                        cont_line = line
                    else:
                        cont_line = line
            elif org_list == []:
                pass
            else:
                cont_line = cont_line + ' ' + line
        else:
            split_line = split_name_speech_count(cont_line)
            org_list.append(split_line)

    ep_df = pd.DataFrame(org_list, columns =['name', 'speech', 'count'])

    return ep_df


def episode_extract(start_ep, end_ep):
    all_ep_df = pd.DataFrame()
    end_ep += 1
    ep_list = list(range(start_ep,end_ep))
    ep_zfill = [str(ep).zfill(3) for ep in ep_list]
    
    for ep in ep_zfill:
        path = ("S1/C1E%s_FINAL.txt" % ep)
        with open(path, "r", encoding="latin-1") as ep_raw:
            ep_raw = ep_raw.read().splitlines()
            ep_df = extract_lines(ep_raw)
            ep_df['episode'] = ep
            all_ep_df = all_ep_df.append(ep_df)

    return all_ep_df

In [88]:
def mapping_names(all_ep_df):
    laura_name_list = ['all', 'laura', 'vex', '-laura']
    matt_name_list = ['all', 'matt', 'matthew', 'decide', '-matt', 'sherri', 'allura', 'gilmore', 'Adra', 'kima', 'kaylee', 'fast', 'uriel', 'balgus', 'mat', 'caught up']
    marisha_name_list = ['all', 'marisha', 'keyleth', '-marisha']
    sam_name_list = ['all', 'sam', 'scanlan', '-sam']
    taliesin_name_list = ['all', 'taliesin', 'percy', '-taliesin']
    liam_name_list = ['all', 'liam', 'vax', '-liam']
    travis_name_list = ['all', 'travis', 'grog', '-travis']
    ashley_name_list = ['all', 'ashley', 'pike', '-ashley']
    orion_name_list = ['orion', 'tiberius', '-orion']
    guest_name_list = ['zac', 'will', 'mary', 'patrick', 'felicia', 'wil', 'chris', 'jon', 'joe', 'darin', 'noelle', 'kit', 'jason', 'patrick rothfuss']
    break_name_list = ['ify', 'dan', 'brian', 'loves in his life', 'ivan', 'kevin', 'this wonderful show are']

    name_list = [laura_name_list, matt_name_list, 
        marisha_name_list, sam_name_list, taliesin_name_list, 
        liam_name_list, travis_name_list, ashley_name_list, 
        orion_name_list, guest_name_list]

    names = ['laura', 'matt', 
        'marisha', 'sam', 'taliesin', 
        'liam', 'travis', 'ashley', 
        'orion', 'guest']

    for names, name_list in zip(names, name_list):
        all_ep_df[names] = all_ep_df['name'].map(lambda y: 1 if any(word in name_list for word in y.split()) else 0)
    
    return all_ep_df

In [89]:
test_df = episode_extract(1, 115)
extract_names = mapping_names(test_df)

In [100]:
extract_names[['name', 'speech', 'count', 'episode']]

,name,speech,count,episode
0,matt,hello everyone my name is matthew mercer,7,001
1,matt,welcome to first episode of critical role and...,65,001
2,so before we get to that,to give you a little backstory on the charact...,104,001
3,grog,right listen up if you have ale then you have...,31,001
4,loves in his life,combat women and ale (chuckle) (record scratch...,194,001
...,...,...,...,...
2209,marisha,that was beautiful,3,115
2210,sam,thanks matt that was really nice,6,115
2211,matt,love you guys,3,115
2212,laura,love you matthew,3,115


In [90]:
extract_names

,name,speech,count,episode,laura,matt,marisha,sam,taliesin,liam,travis,ashley,orion,guest
0,matt,hello everyone my name is matthew mercer,7,001,0,1,0,0,0,0,0,0,0,0
1,matt,welcome to first episode of critical role and...,65,001,0,1,0,0,0,0,0,0,0,0
2,so before we get to that,to give you a little backstory on the charact...,104,001,0,0,0,0,0,0,0,0,0,0
3,grog,right listen up if you have ale then you have...,31,001,0,0,0,0,0,0,1,0,0,0
4,loves in his life,combat women and ale (chuckle) (record scratch...,194,001,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,marisha,that was beautiful,3,115,0,0,1,0,0,0,0,0,0,0
2210,sam,thanks matt that was really nice,6,115,0,0,0,1,0,0,0,0,0,0
2211,matt,love you guys,3,115,0,1,0,0,0,0,0,0,0,0
2212,laura,love you matthew,3,115,1,0,0,0,0,0,0,0,0,0


In [103]:
names = ['laura', 'matt', 
    'marisha', 'sam', 'taliesin', 
    'liam', 'travis', 'ashley', 
    'orion', 'guest']

org_names_df = pd.DataFrame()

for name in names:
    filtered_names_df = extract_names.loc[extract_names[name] == 1]
    filtered_names_df = filtered_names_df[['name', 'speech', 'count', 'episode']]
    filtered_names_df['person'] = name
    org_names_df = org_names_df.append(filtered_names_df)


In [104]:
org_names_df

,name,speech,count,episode,person
12,vex,like so many half-elves vexahlia has spent mo...,210,001,laura
32,laura,hi im laura bailey im also you know a voice a...,16,001,laura
34,laura,and im playing vexahlia the coolest chick ever...,10,001,laura
36,laura,the ranger with the bear named trinket in cas...,20,001,laura
46,laura,yeah we do,3,001,laura
...,...,...,...,...,...
498,mary,thank you,2,115,guest
499,will,luck in battle,3,115,guest
503,mary,thank you,2,115,guest
504,will,bastard treat her well or im going to come back,10,115,guest


In [34]:
grouped_names = test_df.groupby(['name']).agg({'count': ['count', 'sum']})
grouped_names.columns = ['line_count', 'word_count']

In [35]:
total_names = sum(grouped_names['word_count'])
high_word_names = grouped_names.loc[grouped_names['word_count'] >= 1000]
high_word_count = sum(high_word_names['word_count'])
print (high_word_count/total_names)

0.9852446170750576
